In [12]:
# Dependencies
import requests
from pprint import pprint
#from config_om import api_key
#from config_ab import api_key
from config_kr import bls_key
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


#url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"
#url

In [13]:
# Import source data
glassdoor_file = pd.read_csv("./remote_resources/glassdoor/glassdoor.csv")
stackoverflow_file = pd.read_csv("./remote_resources/stackoverflow/2019-survey/survey_results_public_prepped.csv")
h1b1_visas_file = pd.read_csv("./remote_resources/h1b-visas/h1b18.csv")


In [14]:
# Prepare Stack Overflow Survey data for analysis (Filtering to a subset of Columns)
stack_overflow_filtered_columns = stackoverflow_file[['MainBranch','Hobbyist','Employment','Country','Student','EdLevel','EduOther','OrgSize','DevType','YearsCode','YearsCodePro','CareerSat','JobSat','CompTotal','ConvertedComp','WorkWeekHrs','LanguageWorkedWith','LanguageDesireNextYear','DatabaseWorkedWith','DatabaseDesireNextYear','PlatformWorkedWith','PlatformDesireNextYear','Age','Gender','Ethnicity', 'DataScienceDomain'
]]

# Isolate US based surveys, with a Data Science Domain for which the records are complete (i.e. 'NaNs' removed)
stack_overflow_filtered_USOnly = stack_overflow_filtered_columns.loc[stack_overflow_filtered_columns['Country'] == 'United States']
stack_overflow_filtered_DataScienceDomain_USOnly = stack_overflow_filtered_USOnly.loc[stack_overflow_filtered_USOnly['DataScienceDomain'] == True]
stack_overflow_df = pd.DataFrame(stack_overflow_filtered_DataScienceDomain_USOnly.dropna())
stack_overflow_df.count()

MainBranch                1897
Hobbyist                  1897
Employment                1897
Country                   1897
Student                   1897
EdLevel                   1897
EduOther                  1897
OrgSize                   1897
DevType                   1897
YearsCode                 1897
YearsCodePro              1897
CareerSat                 1897
JobSat                    1897
CompTotal                 1897
ConvertedComp             1897
WorkWeekHrs               1897
LanguageWorkedWith        1897
LanguageDesireNextYear    1897
DatabaseWorkedWith        1897
DatabaseDesireNextYear    1897
PlatformWorkedWith        1897
PlatformDesireNextYear    1897
Age                       1897
Gender                    1897
Ethnicity                 1897
DataScienceDomain         1897
dtype: int64

In [15]:
# Define Categories for Analysis
# group by education level
so_groupby_edu = stack_overflow_df.groupby('EdLevel') 

# group by gender
so_groupby_gender = stack_overflow_df.groupby('Gender')

so_groupby_devtype = stack_overflow_df.groupby('DevType')

so_groupby_age = stack_overflow_df.groupby('Age')

so_groupby_emp = stack_overflow_df.groupby('Employment')

so_groupby_mainbranch = stack_overflow_df.groupby('MainBranch')

so_groupby_degree_DevType = stack_overflow_df.groupby(['EdLevel','DevType'] ).agg({'Age': ['count']})

#print(glassdoor_file.head())
#print(stackoverflow_file.head())

In [16]:
print(stack_overflow_df['EdLevel'].unique())
print("-----------------------------------------------"*2)
print(stack_overflow_df['DevType'].unique())

['Master’s degree (MA, MS, M.Eng., MBA, etc.)'
 'Some college/university study without earning a degree'
 'Bachelor’s degree (BA, BS, B.Eng., etc.)'
 'Other doctoral degree (Ph.D, Ed.D., etc.)'
 'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)'
 'Associate degree' 'Professional degree (JD, MD, etc.)'
 'I never completed any formal education' 'Primary/elementary school']
----------------------------------------------------------------------------------------------
['Data or business analyst;Database administrator;Developer, back-end;Developer, desktop or enterprise applications;Developer, front-end;Developer, full-stack;Developer, mobile;DevOps specialist;Engineer, data;Engineer, site reliability;Engineering manager;Senior executive/VP;System administrator'
 'Data or business analyst;Designer;Developer, back-end;Developer, desktop or enterprise applications;Developer, front-end;Developer, full-stack;DevOps specialist;System administrator'
 'Developer, 

In [17]:
so_groupby_mainbranch['Gender'].count()
print(so_groupby_edu['Gender'].count())
print("-----------------------------------------------"*2)
#print(so_groupby_devtype['Gender'].count())
so_survey_edlevel_and_devtype = pd.DataFrame(so_groupby_degree_DevType['Gender'].count()).reset_index()
so_survey_edlevel_and_devtype

EdLevel
Associate degree                                                                       69
Bachelor’s degree (BA, BS, B.Eng., etc.)                                              940
I never completed any formal education                                                  7
Master’s degree (MA, MS, M.Eng., MBA, etc.)                                           477
Other doctoral degree (Ph.D, Ed.D., etc.)                                             145
Primary/elementary school                                                               7
Professional degree (JD, MD, etc.)                                                      7
Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)     38
Some college/university study without earning a degree                                207
Name: Gender, dtype: int64
----------------------------------------------------------------------------------------------


KeyError: 'Gender'

In [24]:
h1b1_visas_file_filtered_columns = h1b1_visas_file[['CASE_STATUS', 'CASE_SUBMITTED', 'DECISION_DATE', 'EMPLOYER_NAME', 'EMPLOYER_CITY', 'EMPLOYER_STATE', 'EMPLOYER_POSTAL_CODE', 'JOB_TITLE', 'SOC_NAME', 'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY', 'WAGE_RATE_OF_PAY_FROM' ]]
h1b1_visas_file_certified = h1b1_visas_file_filtered_columns.loc[h1b1_visas_file_filtered_columns['CASE_STATUS']=='CERTIFIED']
h1b1_visas_file_certified_dropna = h1b1_visas_file_certified.dropna()
h1b1_visas_file_certified_dropna

,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,EMPLOYER_NAME,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,JOB_TITLE,SOC_NAME,FULL_TIME_POSITION,PREVAILING_WAGE,PW_UNIT_OF_PAY,WAGE_RATE_OF_PAY_FROM
0,CERTIFIED,1/29/18,2/2/18,MICROSOFT CORPORATION,REDMOND,WA,98052,SOFTWARE ENGINEER,"SOFTWARE DEVELOPERS, APPLICATIONS",Y,"112,549.00",Year,"143,915.00"
1,CERTIFIED,10/23/17,10/27/17,ERNST & YOUNG U.S. LLP,SECAUCUS,NJ,07094,TAX SENIOR,ACCOUNTANTS AND AUDITORS,Y,"79,976.00",Year,"100,000.00"
2,CERTIFIED,8/30/18,9/6/18,LOGIXHUB LLC,IRVING,TX,75062,DATABASE ADMINISTRATOR,DATABASE ADMINISTRATORS,Y,"77,792.00",Year,"78,240.00"
4,CERTIFIED,8/31/18,9/7/18,"ECLOUD LABS,INC.",ISELIN,NJ,08830,MICROSOFT DYNAMICS CRM APPLICATION DEVELOPER,"SOFTWARE DEVELOPERS, APPLICATIONS",Y,"87,714.00",Year,"95,000.00"
5,CERTIFIED,5/22/18,5/29/18,OBERON IT,IRVING,TX,75038,SENIOR SYSTEM ARCHITECT,"SOFTWARE DEVELOPERS, APPLICATIONS",Y,"71,864.00",Year,"74,000.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
654355,CERTIFIED,3/21/18,3/27/18,CROCKETT TECHNICAL LLC,IRVING,TX,75062,ASSOCIATE SOFTWARE ENGINEER,"SOFTWARE DEVELOPERS, APPLICATIONS",Y,"90,646.00",Year,"91,000.00"
654356,CERTIFIED,3/5/18,3/9/18,BRUNSWICK CORPORATION,METTAWA,IL,60045,MANAGER GLOBAL SUPPLY CHAIN BUSINESS ANALYTICS,INDUSTRIAL ENGINEERS,Y,"68,598.00",Year,"74,000.00"
654357,CERTIFIED,11/13/17,11/17/17,JOFEMAR USA INC.,DORAL,FL,33172,BUSINESS RESEARCH AND DEVELOPMENT,MARKET RESEARCH ANALYSTS AND MARKETING SPECIAL...,Y,"65,936.00",Year,"65,936.00"
654358,CERTIFIED,3/21/18,3/27/18,IBM INDIA PRIVATE LIMITED,DURHAM,NC,27709,PACKAGE SOLUTION CONSULTANT,COMPUTER SYSTEMS ANALYSTS,Y,"78,416.00",Year,"81,837.00"


In [24]:
#query = "data science"
#query
#query_url = url + "api-key=" + api_key + "&q=" + query
#query_url

In [20]:
#articles = requests.get(query_url).json()
#articles

{'status': 'OK',
 'copyright': 'Copyright (c) 2020 The New York Times Company. All Rights Reserved.',
 'response': {'docs': [{'abstract': 'A new California privacy law gives consumers the right to see and delete their data. But getting access often requires giving up more personal details.',
    'web_url': 'https://www.nytimes.com/2020/01/15/technology/data-privacy-law-access.html',
    'snippet': 'A new California privacy law gives consumers the right to see and delete their data. But getting access often requires giving up more personal details.',
    'lead_paragraph': 'The new year ushered in a landmark California privacy law that gives residents more control over how their digital data is used. The Golden State isn’t the only beneficiary, though, because many companies are extending the protections — the most important being the right to see and delete the personal data a company has — to all their customers in the United States.',
    'print_section': 'B',
    'print_page': '6',
 

In [21]:
#articles_list = articles["response"]["docs"]
#pprint(articles_list)

[{'_id': 'nyt://article/15cdaaf4-10c0-54b3-a439-7ca17ee0a349',
  'abstract': 'A new California privacy law gives consumers the right to see '
              'and delete their data. But getting access often requires giving '
              'up more personal details.',
  'byline': {'organization': None,
             'original': 'By Kashmir Hill',
             'person': [{'firstname': 'Kashmir',
                         'lastname': 'Hill',
                         'middlename': None,
                         'organization': '',
                         'qualifier': None,
                         'rank': 1,
                         'role': 'reported',
                         'title': None}]},
  'document_type': 'article',
  'headline': {'content_kicker': None,
               'kicker': None,
               'main': 'Want Your Personal Data? Hand Over More Please',
               'name': None,
               'print_headline': '‘Privacy Circle of Hell’: To Get Your '
                           

                  'legacy': {},
                  'rank': 0,
                  'subType': 'facebookJumbo',
                  'subtype': 'facebookJumbo',
                  'type': 'image',
                  'url': 'images/2020/01/15/business/15datarequest-promo/15datarequest-promo-facebookJumbo.jpg',
                  'width': 1050},
                 {'caption': None,
                  'credit': None,
                  'crop_name': 'watch308',
                  'height': 348,
                  'legacy': {},
                  'rank': 0,
                  'subType': 'watch308',
                  'subtype': 'watch308',
                  'type': 'image',
                  'url': 'images/2020/01/15/business/15datarequest-promo/15datarequest-promo-watch308.jpg',
                  'width': 312},
                 {'caption': None,
                  'credit': None,
                  'crop_name': 'watch268',
                  'height': 304,
                  'legacy': {},
                  'rank'

                  'type': 'image',
                  'url': 'images/2019/12/19/opinion/19warzelnewsWeb/merlin_165771540_b3c1ce62-71c0-4a0a-9748-786709fc6072-blogSmallInline.jpg',
                  'width': 151},
                 {'caption': None,
                  'credit': None,
                  'crop_name': 'mediumFlexible177',
                  'height': 118,
                  'legacy': {},
                  'rank': 0,
                  'subType': 'mediumFlexible177',
                  'subtype': 'mediumFlexible177',
                  'type': 'image',
                  'url': 'images/2019/12/19/opinion/19warzelnewsWeb/merlin_165771540_b3c1ce62-71c0-4a0a-9748-786709fc6072-mediumFlexible177.jpg',
                  'width': 177},
                 {'caption': None,
                  'credit': None,
                  'crop_name': 'videoSmall',
                  'height': 281,
                  'legacy': {},
                  'rank': 0,
                  'subType': 'videoSmall',
        

                  'crop_name': 'tmagArticle',
                  'height': 888,
                  'legacy': {},
                  'rank': 0,
                  'subType': 'tmagArticle',
                  'subtype': 'tmagArticle',
                  'type': 'image',
                  'url': 'images/2014/08/27/technology/27bits-etzioni/27bits-etzioni-tmagArticle.jpg',
                  'width': 592},
                 {'caption': None,
                  'credit': None,
                  'crop_name': 'jumbo',
                  'height': 1024,
                  'legacy': {},
                  'rank': 0,
                  'subType': 'jumbo',
                  'subtype': 'jumbo',
                  'type': 'image',
                  'url': 'images/2014/08/27/technology/27bits-etzioni/27bits-etzioni-jumbo.jpg',
                  'width': 683},
                 {'caption': None,
                  'credit': None,
                  'crop_name': 'blog225',
                  'height': 338,
            

                  'crop_name': 'master675',
                  'height': 519,
                  'legacy': {},
                  'rank': 0,
                  'subType': 'master675',
                  'subtype': 'master675',
                  'type': 'image',
                  'url': 'images/2013/11/12/business/12bits-data/12bits-data-master675.jpg',
                  'width': 675},
                 {'caption': None,
                  'credit': None,
                  'crop_name': 'master180',
                  'height': 138,
                  'legacy': {},
                  'rank': 0,
                  'subType': 'master180',
                  'subtype': 'master180',
                  'type': 'image',
                  'url': 'images/2013/11/12/business/12bits-data/12bits-data-master180.jpg',
                  'width': 180},
                 {'caption': None,
                  'credit': None,
                  'crop_name': 'popup',
                  'height': 499,
                  'lega

                  'width': 362},
                 {'caption': None,
                  'credit': None,
                  'crop_name': 'slide',
                  'height': 352,
                  'legacy': {},
                  'rank': 0,
                  'subType': 'slide',
                  'subtype': 'slide',
                  'type': 'image',
                  'url': 'images/2015/01/27/technology/27bits-debt/27bits-debt-slide.jpg',
                  'width': 600},
                 {'caption': None,
                  'credit': None,
                  'crop_name': 'superJumbo',
                  'height': 1200,
                  'legacy': {},
                  'rank': 0,
                  'subType': 'superJumbo',
                  'subtype': 'superJumbo',
                  'type': 'image',
                  'url': 'images/2015/01/27/technology/27bits-debt/27bits-debt-superJumbo.jpg',
                  'width': 2048},
                 {'caption': None,
                  'credit': None,


  'lead_paragraph': '',
  'multimedia': [],
  'news_desk': 'None',
  'print_page': '26',
  'pub_date': '1966-07-30T05:00:00+0000',
  'section_name': 'Archives',
  'snippet': '',
  'source': 'The New York Times',
  'type_of_material': 'Archives',
  'uri': 'nyt://article/7f1abebe-df7b-57c2-a799-f506e2023b0e',
  'web_url': 'https://www.nytimes.com/1966/07/30/archives/mohawk-data-sciences.html',
  'word_count': 0},
 {'_id': 'nyt://article/41d1cebe-9a12-5da3-b1c5-450e4bcf4d41',
  'abstract': 'The Medaphis Corporation, a provider of business management '
              'services to the health care industry, agreed yesterday to '
              'acquire the Health Data Sciences Corporation in a stock deal '
              'worth about $255.2 million. Under the agreement, Atlanta-based '
              'Medaphis will offer 6.13 million shares of its common stock for '
              'Health Data Sciences in a pooling of interests. Medaphis will '
              'also assume or issue stock options fo

In [22]:
print("Your Reading List")
for article in articles_list:
    print(article["web_url"])

Your Reading List
https://www.nytimes.com/2020/01/15/technology/data-privacy-law-access.html
https://www.nytimes.com/2019/12/19/opinion/tracking-phone-data.html
https://bits.blogs.nytimes.com/2013/04/07/the-potential-and-the-risks-of-data-science/
https://bits.blogs.nytimes.com/2014/08/27/looking-to-the-future-of-data-science/
https://bits.blogs.nytimes.com/2015/04/28/less-noise-but-more-money-in-data-science/
https://bits.blogs.nytimes.com/2013/11/12/program-seeks-to-nurture-data-science-culture-at-universities/
https://bits.blogs.nytimes.com/2015/01/27/a-data-science-lender-takes-on-student-debt/
https://www.nytimes.com/2013/04/14/education/edlife/universities-offer-courses-in-a-hot-new-field-data-science.html
https://www.nytimes.com/1966/07/30/archives/mohawk-data-sciences.html
https://www.nytimes.com/1996/05/25/business/company-news-medaphis-set-to-add-health-data-sciences.html
